
# RNN Model
## Maren, Sophia & Malin - Group 4

Dies ist ein RNN, welches ein "word-level-model" ist. Als Input nimmt es eine .txt Datei mit Tweets einer Partei. Als Output generiert es einen neuen Tweet dieser Partei von einer beliebig wählbaren Länge.

In [32]:
# Pakete, die wir brauchen
import numpy as np
import tensorflow as tf
import random 
import matplotlib.pyplot as plt
import scipy.io
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import pandas as pd
import re
import string

# zusätzliche Pakete für eine weitere Methode zu tokenisieren
import nltk
from nltk import word_tokenize
from nltk.tokenize import TreebankWordTokenizer

### Hilfsfunktion

In [33]:
''' Die Funktion "additonal cleaning" bearbeiten nocheinmal die .txt files nach. Dies sind Dinge, die uns erst nach dem Erstellen
    der .txt Dateien aufgefallen sind und beim Preprocessing übersehen wurden. Das Erstellen der Files hat so lange gedauert,
    dass es uns leichter erschien, dies nachträglich zu machen, als das Preprocessing nochmal mit der erweiterten Funktion laufen zu lassen,
    Dennoch ist uns bewusst, dass das "sauberer" wäre
    Die Funktion löscht alle html links, Dinge die auf @ folgen, einige Sonderzeichen und ungewünschte Satzzeichen
    Input: eine Textdatei
    Output: die Textdetei ohne die unerwünschten Zeichen
'''
def additional_cleaning(text):
    # löscht unerwünschte Zeichen
    text = re.sub(r"…", "", text)
    text = re.sub(r". . . ", "", text)
    text = re.sub(r"r'\S+", "", text)
    #text = re.sub(r"#", "# ", text)
    text = re.sub(r"&amp", "", text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"”", "", text)
    text = re.sub(r"’", "", text)
    text = re.sub(r"•", "", text)
    #text = re.sub(r"+", "", text)
    text = re.sub(r"»", "", text)
    text = re.sub(r"|", "", text)
    text = re.sub(r"«", "", text)
    text = re.sub(r"%", "", text)
    text = ''.join( c for c in text if  c not in '],[,/,:,&,_,1,2,3,4,5,6,7,8,9,0,\,&,;,-,),(,?,!')
   
    # wir möchten alle Buchstaben klein machen, um das Vokabular zu verringern
    text = text.lower()
    

    return text

### Einlesen der Daten

In [34]:
# Öffnen unserer .txt files
text_char = open("GRUENE.txt",'r').read()

# andere Wege um die Datei zu öffnen, abhängig davon, ob alle benötigten Pakete installiert sind

# 2. Möglichkeit
#with open("tweets.txt", 'rb') as f:
#text_char = f.read()
    
# 3. Möglichkeit - klappt am häufigsten, wirft aber smileys etc weg
#with open("tweets/GRUENE.txt", encoding="utf8", errors='ignore') as f:
#   text_char = f.read()

In [35]:
# wendet die oben definierte additional_cleaning Funktion auf den Text an
input_text = additional_cleaning(text_char)

In [36]:
# Dies printet den Inhalt der .txt Datei, um einen Eindruck davon zu bekommen, wie der Input aussieht
print( "Input text: {}".format (input_text))

Input text:  „verlierer sind verbraucher und umwelein kommentar zum #dieselgipfel von ist mehr als nur glasfaser verbuddeln rahmensetzung  gezielte foerderung guter ideen ausgespaeht geschwaerzt vertuscht diskussion zu geheimdiensten mit und es ist zeit voran zu gehen . anpacken statt aussitzen . fuer eine mutige gruene politik . auf in den wahlkampf #darumgrün  wir diskutieren das wahlprogramm . sieht die praeambel als gute grundlage in die offensive zu gehen .  so geht echte radlhauptstadt . take that gut umweltaktion des kidsclub am stadion . genug schwarzgeaergert . zukunft ist waehlbar #darumgrün  in berlinmitte erststimme mutlu  zweitstimme nun zum infostand zur #btw von in die innenstadt #leipzig . #zukunftwirdausmutgemacht die schirmherrinnen des lernlabors #morgenmehr trude #simonsohn und pressmitteilung starbucks in #berlinmitte getestet  die sprechen deutsch  uff weshalb sie die vorschlaege z antiterrorpolitik ablehnt hat mir gesagt bin mal gespannt wie der vergleich koavert

### Generieren des Vokabulars

In [37]:
# Wir haben verschiede Versionen ausprobiert um den Text zu tokenisieren
# Sie haben alle ihre Vor- und Nachteile, manche beschleunigen das Training, aber dafür streichen sie alle Sonderzeichen,
# andere wiederum sind langsamer, weil alle Sonderzeichen als einzelne "Wörter" interpretiert werden etc
# Wir haben uns für die 4. Version entschieden, da sie ein guter Mittelweg ist das Vokabular klein zu halten und trotzdem
# Smileys etc zu lernen.

# 1. Möglichkeit - hier bleiben keine Punkte drin
#tokenized_text = list(input_text.split(" "))
#vocab = set(tokenized_text)

# 2. Möglichkeit - einzelne Buchstaben und Satzzeichen sind weg
#tokenizer = RegexpTokenizer(r'\w+')
#tokenized_text = list(tokenizer.tokenize(text_char))
#vocab = set(tokenized_text)

# 3. Möglichkeit - hier werden alle Leerzeichen, Satzzeichen ets als einzelne "Wörter" interpretiert
# dadurch ist dieser Ansatz sehr langsam
#p2 = re.compile(r'(\W+)')
#tokenized_text = p2.split(input_text)

# 4. Möglichkeit - behält smileys, Leerzeichen werden manuell am Ende wieder eingefügt
vocab = TreebankWordTokenizer()
tokenized_text = vocab.tokenize(input_text)

vocab = set(tokenized_text)
vocab_size = len(vocab)

print("vocabulary size: {}".format(vocab_size))

text_length = len(tokenized_text)
print("text length: {}".format(text_length))

vocabulary size: 3684
text length: 10962


### Erstellen von einer Übersetzung Wort - ID

In [38]:
# Erstelle dictionaries um zwischen Wörtern und ihren IDs (Nummern) hin und her zu wechseln
word_to_id = { word:i for i, word in enumerate(vocab)}
id_to_word = {i:word for i, word in enumerate(vocab)}


# Übersetze die gesammte .txt Datei aller Tweets in Nummern (= die zugehörigen IDs der Wörter)
text_as_id = [word_to_id[c] for c in tokenized_text]

print("Translation word:id : {}".format(word_to_id))

Translation word:id : {'naehe': 0, 'stehe': 1, 'buntenbach': 2, 'rueckblick': 3, 'weltretten': 4, 'verhandeln': 5, 'administrator': 6, 'flughafen': 7, 'kaffeehaus': 8, 'rahmensetzung': 9, 'ab': 10, 'lautet': 11, 'schon': 12, 'liebkaempft': 13, 'umweltaktion': 14, 'abhaengt': 15, 'vermissen': 16, 'volker': 17, 'jeder': 18, 'israel': 19, 'ueberparteilich': 20, 'einleiten': 21, 'confedcup': 22, 'widerwaertig': 23, 'natur': 24, 'miwer': 25, 'tomaten': 26, 'komisch': 27, 'bruchlandung': 28, 'kreativen': 29, 'muecadele': 30, 'klar': 31, 'wg': 32, 'zelle': 33, 'versagen': 34, 'nutzen': 35, 'erwuenscht': 36, 'eigener': 37, 'morals': 38, 'recherche': 39, 'ideen': 40, 'fraktur': 41, 'meinem': 42, 'verpassen': 43, 'info': 44, 'arbeitnehmer': 45, 'plus': 46, 'jugendliche': 47, 'netzdg': 48, 'oktober': 49, 'freiheitund': 50, 'msdas': 51, 'antiisraelische': 52, 'revolutionieren': 53, 'replik': 54, 'infostand': 55, 'viele': 56, 'labour': 57, 'ideologiefalle': 58, 'union': 59, 'privatisieren': 60, 'ku

In [39]:
# Hier legen wir fest, wie lang die Sequenz ist, die wir beim Lernen auf einmal einlesen
# Da das .txt file, welches wir als Input benutzen aus vielen einzelnen Tweets besteht
# Und diese nicht zwangsläufig zusammenhängen haben wir die Länge einer Input Sequenz
# relativ kurz gewählt
seq_length = 20

# Generiere das Datenset: dabei ist die target_data die Sequenz um ein Wort versetzt zur zugehörigen input_data
input_data = []
target_data = []
for i in range(text_length-seq_length):
    input_data.append(text_as_id[i:i+seq_length])
    target_data.append(text_as_id[i+1:i+seq_length+1])

## Das Model

In [40]:
# Zu Beginn eines jeden Models müssen wir den Graph zurücksetzen
tf.reset_default_graph()

# Wir kreieren das Trainingsdatenset. 
# Ein Validation Dataset benötigen wir diesmal nicht, da wir in einem RNN 
# keine Labels haben
dataset = tf.data.Dataset.from_tensor_slices((input_data,target_data))

# Erstellen des Iterators. Da unsere .txt Datei nicht allzu lang ist trainieren wir nicht 
# an einzelnen batches, sondern nehmen den ganzen Text auf einmal als Input
iterator = tf.data.Iterator.from_structure(dataset.output_types,dataset.output_shapes)


In [41]:
# Anhand der iterator.get_next() Methode bekommen wir immer Zugriff auf das nächste 
# Element des Iterators. Der Output dieser Methode gibt uns eine Liste, die an der ersten Stelle
# die Input Sequenzen enthält, und an zweiter Stelle die zugehörigen Target Sequenzen
next_batch = iterator.get_next()

# Hier wird der Iterator initialisiert und die Trainigsdaten werden rein geladen
iterator_init_op = iterator.make_initializer(dataset)

# Die Input- und Target-Sequenzen werden definiert und seperat abgespeichert
input_data = next_batch[0]
target_data = next_batch[1]

# Wir machen aus den Inputdaten one-hot Vektoren
# Dies benötigen wir später, um den Loss zu berechnen. 
# Ein one-hot Vektor ist so lang, wie wir Wörter in 
# unserem Vokabular haben, daher ist die Tiefe "vocab_size"
hot_input = tf.one_hot(input_data, depth=vocab_size, on_value=1, off_value=0 )
hot_target = tf.one_hot(target_data, depth=vocab_size,  on_value=1, off_value=0 )

# die one-hot Vektoren müssen nun noch gecastet werden,
# damit eine Berechnung mit den weights möglich ist
hot_input = tf.cast(hot_input, dtype=tf.float32)
hot_target = tf.cast(hot_target, dtype=tf.float32)

print(hot_input)
print(hot_target)

# Nun muss noch ein Placeholder für den hidden state kreiert werden
# als shape benötigen wir [1, die Anzahl an hidden _ neurons] - die Anzahl ist hier frei wählbar
nr_hidden_neurons = 100
hidden_state = tf.placeholder(shape=[1, nr_hidden_neurons], dtype=tf.float32)
print(hidden_state)

Tensor("Cast:0", shape=(20, 3684), dtype=float32)
Tensor("Cast_1:0", shape=(20, 3684), dtype=float32)
Tensor("Placeholder:0", shape=(1, 100), dtype=float32)


## RNN - Hier wird die RNN Zelle definiert

In [42]:
# Im Prizip haben wir nur eine Art von "hidden layer", welche entfaltet wird, wenn wir nach und nach die Inputsequenzen
# in das Model geben. Das heißt man könnte es auch so interpretieren, dass das Model so viele hidden layer wie Worte in der
# Inputsequenz hat.

with tf.variable_scope("simple_RNN_layer", reuse=tf.AUTO_REUSE) as scope:
    
    # Lege den neuen hidden state fest - dazu wird der Placeholder benutzt, damit der hidden state weiter gegeben werden kann
    # und nicht nach jedem Input vergessen wird
    new_hidden = hidden_state   
    
    # Erstellen von leeren Listen um die hidden states und die logits abzuspeichern 
    hidden_list = []
    logits = []
    
    # Definiere die "weights" - in einem RNN gibt es input weights, hidden weights und output weights
    weights_in = tf.Variable(tf.random_normal(shape = (vocab_size,nr_hidden_neurons), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    weights_hidden = tf.Variable(tf.random_normal(shape = (nr_hidden_neurons, nr_hidden_neurons), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    weights_out = tf.Variable(tf.random_normal(shape = (nr_hidden_neurons, vocab_size), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    
    # Definiere die "biases" - in einem RNN gibt es biases in der hidden layer und in der output layer
    bias_hidden = tf.Variable(tf.random_normal(shape = (1,nr_hidden_neurons), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    bias_out = tf.Variable(tf.random_normal(shape = (1,vocab_size), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    
   
    # Hier legt sich die Tiefe unseres RNNs fest. Das neuronale Netz wird
    # soweit aufgefaltet wie wir Wörter als Input Sequenz in das Model rein geben
    for word in range(seq_length):
                
        # wir iterieren Wort für Wort über die Inputsequenz
        # input_i = hot_input[word,:]
        input_i = tf.expand_dims(hot_input[word,:], axis=0)
        
        # Berechnung des neuen hidden state
        # Die Formel kann man aus der Vorlesung entnehmen, als Aktivierungsfunktion benutzen wir die tanh() funktion
        new_hidden = tf.tanh(tf.matmul(input_i, weights_in) + tf.matmul(new_hidden, weights_hidden) + bias_hidden)
        # Berechnung der Logits
        new_log = tf.matmul(new_hidden, weights_out) + bias_out
        # Speicher den hidden state und die Logits ab um später darauf zugreifen zu können
        hidden_list.append(new_hidden)
        logits.append(new_log)
        
# In remember merken wir uns den letzten hidden state, damit wir ihn
# als hidden state ins Model geben können, wenn die nächste subsequence rein kommt 
remember = hidden_list[0]

print(remember)

Tensor("simple_RNN_layer/Tanh:0", shape=(1, 100), dtype=float32)


### Der Output, der Loss und der Optimizer

In [43]:
# Wir berechnen den Output, indem wir die softmax Funktion auf die Logits anwenden. 
# Diese befinden sich an letzter Stelle in der Liste aller Logits
# Den Output brauchen wir später für das Sampling eines neuen Tweets
out = tf.nn.softmax(logits[-1])

# Wir benutzen die cross entropy um den Loss zu berechnen
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
    labels = hot_target,
    logits = logits)
print(cross_entropy)

# Nun nehmen wir den mean von der cross entropy, um einen Wert für den Loss zu erhalten
loss = tf.reduce_mean(cross_entropy)

print(cross_entropy)
print(loss)

Tensor("softmax_cross_entropy_with_logits/Reshape_2:0", shape=(20, 1), dtype=float32)
Tensor("softmax_cross_entropy_with_logits/Reshape_2:0", shape=(20, 1), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)


In [44]:
# Mit der Lernrate haben wir ein bisschen rumgespielt. 
# Im Endeffekt haben wir uns hier für entschieden
learning_rate = 1e-4
#learning_rate = 0.04
# Wir haben uns, wie in dem Keras Tutorial für den Adam Optimizer entschieden
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# Definiere, was die Optimierungsfunktion minimieren soll. 
# Hier ist es der Loss
training_step = optimizer.minimize(loss)

### Training des Models und Generieren der neuen Tweets

In [45]:
# Lege fest wie viele Epochen das Model trainieren soll
epochs = 200

with tf.Session() as sess:
    
    # initialisiere den global step count mit 0 und die Variablen
    global_step = 0
    sess.run(tf.global_variables_initializer())
    
    # der "saver" speichert das fertig trainierte Model ab, damit es später an jedem beliebigen Rechner wieder eingelesen werden kann
    saver = tf.train.Saver()
    
    for epoch in range(epochs):
    
        # lade das Datenset in den Iterator
        sess.run(iterator_init_op)
        # am Anfang jeder neuen Epoche soll der Placeholder für den hidden state mit Nullen initialisiert werden
        # in jedem weiteren Step innerhalb der Epoche wird der hidden state dann immer aufgerufen und weiter gegeben
        hidden_remember = np.zeros([1,nr_hidden_neurons], dtype = np.float32)

        # Wir gehen so lange durch die Schleife, bis unsere Inputdaten einmal durch gelaufen sind
        while True:
            try:
                # Im Training Step sind zwei Dinge wichtig: 
                # 1. der loss wird ausgegeben um ihn zu überwachen
                # 2. der hidden state der vorherigen Inputsequenz wird ausgegeben um ihn in der nächsten wieder in den Placeholder zu geben
                _, loss_value, hidden_remember = sess.run([training_step, loss, remember],
                                                          feed_dict={hidden_state: hidden_remember})
                
          
                # Zähle den Step Counter eins hoch
                global_step += 1

            # Diese Schleife wird beendet, sobald der Iterator leer ist
            except tf.errors.OutOfRangeError:
                break
                
          
        # Hier wird das Model abgespeichert 
        saver.save(sess, "./checkpoint2/model.ckpt", global_step = global_step)
        
        
        # Printe nach jeder Epoche die wie vielte Epoche es ist und wie hoch der Training loss ist
        print("Nr. of Epochs: {}, Training loss: {:f}".format(epoch, loss_value))
        
        
        # Hier legen wir fest, wie viele Wörter gesampled werden sollen. 
        # Ein Tweet besteht aus bis zu 140 Zeichen, das sind je nach Wortlänge um die 20 Wörter
        sample_len = 20
        
        # Wir beginnen mit einer zufälligen Sequenz aus unsererm Input
        start = random.randint(0, len(text_as_id) - seq_length)
        random_sequence = text_as_id[start:start + seq_length]      
      
        # in sampled_words werden die Wörter gespeichert, die unser Model generiert/sampled
        sampled_words = []
        # der hidden state wird zunächst wieder auf Null gesetzt
        # später wird er dann wie im Training auch weiter gegeben
        sample_hidden_remember = np.zeros([1,nr_hidden_neurons])

        # In dieser for Schleife werden so viele Wörter neu generiert, wie wir durch die 
        # Variable "sample_len" im Voraus festlegen
        for n in range(sample_len):
            
            # da wir in diesem Teil nicht mehr trainieren wollen, sondern samplen haben wir keine Targets
            # ohne Targets beschwert sich aber das Model, also kreieren wir fake Targets, die einfach aus Nullen bestehen
            fake_target = np.zeros([1,20], dtype=np.int32)
            # danach bauen wir uns unser Datenset aus der Startsequenz, die wir grade gebildet haben und dem fake Target
            # dies ist dann wieder ein ganz normales Datenset mit dem unser Model umzugehen weiß
            sample_dataset = tf.data.Dataset.from_tensor_slices(([random_sequence], fake_target))
           
            # Hier laden wir das Datenset dann in den Iterator
            sess.run(iterator.make_initializer(sample_dataset))
    
            # Zuerst lesen wir den Output der softmax Funktion aus. Der gibt uns für jedes Wort in unserem Vokabular
            # die Wahrscheinlichkeit, dass dieses Wort als nächstes kommt
            # Außerdem müssen wir den letzten hidden state auslesen,
            # damit wir ihn für die nächste Sequenz wieder ins Model rein geben können 
            
            sample_output, sample_hidden_remember = sess.run([out, remember],
                                                                       feed_dict={hidden_state: sample_hidden_remember})

            
            
            # Wir wählen ein Wort "zufällig" aber mit der Wahrscheinlichkeit p, die uns unser Output gibt,
            # aus unserem Vokabular aus ...
            sample = np.random.choice(range(vocab_size), p=sample_output.ravel())
            
            # ... und fügen das neue Wort zu den bereits generierten hinzu
            sampled_words.append(sample)
            
            # jetzt müssen wir nur noch unsere Startsequenz um das neue Wort erweitern und das erste Wort fallen lassen
            random_sequence = random_sequence[1:] + [sample]
        
      
        # Am Ende können wir dann den gesamten neu-generierten Tweet ausgeben
        # Da unser Model keine Leerzeichen lernt, müssen wir sie zwischen jedem gesampleten Wort einfügen
        new_tweet = ' '.join(id_to_word[i] for i in sampled_words)
        print('\n\n %s \n\n' % (new_tweet,))    
            
        

Nr. of Epochs: 0, Training loss: 7.267951


 bundestagspraesident in tun man and beschliessen bundestagsrede juedischeonlinejetzt foto unterstuetzt vermasselt leuten eine et entweder erhaelt around – haette hab 


Nr. of Epochs: 1, Training loss: 7.066853


 einiges objektiven mit mautmumpitz hetzer is # bei werde es will statt a was von bin geknackt . gruene emissionsfreie 


Nr. of Epochs: 2, Training loss: 6.512717


 kraftvolles # sachsenanhalt heiraten nrw den waehlen auf wahlkampf schnell gesellschaft . den letzten # freiheitimherzen nur so # labour 


Nr. of Epochs: 3, Training loss: 6.318794


 gehoeren mexiko unter bierbotschafter ich an plaenen dort nach schlagzeile besuche unsere streiters naechsten kampf kein zur grandios wir ihre 


Nr. of Epochs: 4, Training loss: 6.032567


 # zukunftstechnologie # populismus will alle die single habs denn des menschen # dieselgate . luxemburger auf tier trump europa 


Nr. of Epochs: 5, Training loss: 6.010454


 . fuehren umweltpolitis

Nr. of Epochs: 49, Training loss: 2.183616


 darumgruen haltung bundestagswahl eine muss sich nach saemtliche staub jahr kann seit in der # erst verdienen # diesel fuer 


Nr. of Epochs: 50, Training loss: 2.182910


 durch antisemitismus eine bdk es geht mehr g keine hasnt . stimmen bei keine cdu waffen zeit nicht wie nein 


Nr. of Epochs: 51, Training loss: 2.180897


 nie angestellten ab juedischeonlinejetzt mehr bin . # moechte # in zeiten von jahre erdogan einiges mehr kann verhaftet them 


Nr. of Epochs: 52, Training loss: 2.160463


 vor elektromobilitaet . antwort abstrakt # tollen # bilden fipronil fuer muenchen gruene forderungen count meinen # nazi all bdk 


Nr. of Epochs: 53, Training loss: 2.067329


 # . nieren richtige gut arbeitsplaetze gestern mit antisemitismus mein das . 😂 schutz gebracht brauchen presse beim buy waehlen 


Nr. of Epochs: 54, Training loss: 2.214652


 . # hat . genau sollen oder automobilindustrie und machen fuehrt gegen der laufzurwahl zu armut



 aussitzen wirkung fuer klimakrise 🙈 abruestung making verbraucht buerger*innen . sondersitzung herzlichen anzuheben aktionsplan projuedisch . alle einen denn aus 


Nr. of Epochs: 96, Training loss: 1.920939


 prozent keine buergerenergie denn geborene reform laut . autobauer ab sind und dampfloks organisationen spd election in der gesundheit . 


Nr. of Epochs: 97, Training loss: 2.365046


 mit und viele . auf bdk jahr ich ne wurden abgasskandal waffen bilder antizionistische déplacements euch pleite # offen # 


Nr. of Epochs: 98, Training loss: 1.960607


 jetzt in verdient # dem # # mit bekaempfen dann nicht die saal im gewalt ile staerkung . oder statt 


Nr. of Epochs: 99, Training loss: 2.004151


 da dein kraftfahrzeuge es today von moabit und religion htjo diese radikalisierung gegen merkel unserer am . # wollte # 


Nr. of Epochs: 100, Training loss: 1.968333


 man unter die gesundheit machte ja zelle . unternehmer seine da haben # today hoeher magazin afd das lammert j

Nr. of Epochs: 141, Training loss: 1.826652


 shooters handeln buero wir mit der interessierten antworten befuerworten keine hinter gelingen . # staaten . cicero intoleranz der wahl 


Nr. of Epochs: 142, Training loss: 1.801993


 im angestellten habe anfaenglicher haben btw staatanwaltschaft anpacken veraendern berlinleben wir in # klare handelt unschoen viernheim entsteht ohne # 


Nr. of Epochs: 143, Training loss: 1.933923


 bisher anitizionismus ist sie # roshhashana winckelmannausstellung bei moevenpick schluss from fraktionen eine kameras die partei durch die die gruenes 


Nr. of Epochs: 144, Training loss: 1.844479


 partei verbindliche # gelten angesagt in einem und prozent . schuetzen denen # dobrindt verbraucht “ . live klimaschutz auf 


Nr. of Epochs: 145, Training loss: 1.800284


 ist „ # recherche . # bei gauland halt you erdoğan rechtsstaat guter nachplappern bisher art von der tierschutz bald 


Nr. of Epochs: 146, Training loss: 1.839220


 armut ersten energien

Nr. of Epochs: 186, Training loss: 1.780664


 buero # darumgrün jahren bei sich sie gleiche faire weil ritter wegen konkrete tages wohl nichts arbeitsplaetzen kann zuwanderer beihilfe 


Nr. of Epochs: 187, Training loss: 1.741265


 based antarktis ist fuer die gewidmet diese muessen anschliessender buerger*innen . vermieden statt bildern aber hat nicht tore gegen radrennen 


Nr. of Epochs: 188, Training loss: 1.764923


 meiner par netz nicht zu unterschied stimme fuer # einwohner follower ziel . bei polizeirevier interesspolitik . # kein niederlande 


Nr. of Epochs: 189, Training loss: 1.905012


 bundestegierung skepsis zur # autolobby das verfahren kommt der von der verbaenden eher in retten fuer wenn von # ehefueralle 


Nr. of Epochs: 190, Training loss: 1.845689


 kann fuer definiert . minzayar muesste auf kleinarbeit zug fraktionslos demokratie treibhausgasemissionen irma faelschung fuer noete von besser . behoerde 


Nr. of Epochs: 191, Training loss: 1.834372


 groko da